# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env", override=True)

True

In [44]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"

langsmith_api_key = os.getenv("LANGCHAIN_API_KEY")
if langsmith_api_key:
    print("LANGCHAIN_API_KEY is present")
else:
    print("LANGCHAIN_API_KEY is NOT present, paste key at the prompt:")
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

LANGCHAIN_API_KEY is present


We'll also want to set a project name to make things easier for ourselves.

In [7]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [8]:
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print("OPENAI_API_KEY is present")
else:
    print("OPENAI_API_KEY is NOT present, paste key at the prompt:")
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OPENAI_API_KEY is present


## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [9]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [10]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/home/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/home/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/home/AIE7/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [11]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [12]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [ ]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

We can save and load our knowledge graphs as follows.

In [14]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 39, relationships: 476)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, 
MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


#### ✅ Answer:

- `SingleHopSpecificQuerySynthesizer`: this type of query synthesizer generates questions from a single document, and hence answerable from a single document or chunk
- `MultiHopAbstractQuerySynthesizer`: this type of query synthesizer generates questions from multiple document and is broad in its focus, i.e. it requires reasoning (abstract/higher level) across multiple documents with a conceptual link between them
- `MultiHopSpecificQuerySynthesizer`: this type of query synthesizer generates questions from multiple document and focusses on specific queries, i.e. it requires reasons on specific facts across multiple documents with a conceptual link between them

Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How do academic calendars impact compliance wi...,"[Chapter 1 Academic Years, Academic Calendars,...",Academic calendars are essential for ensuring ...,single_hop_specifc_query_synthesizer
1,What is Volume 8 and how does it relate to cli...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of Clinical Work in a Standard Term ...,single_hop_specifc_query_synthesizer
2,Could you explain how the Federal Work-Study (...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
3,Pell Grant how does it work?,[both the credit or clock hours and the weeks ...,The Pell Grant amount a student can get depend...,single_hop_specifc_query_synthesizer
4,How does Title IV regulation impact disburseme...,[Disbursement Timing in Subscription-Based Pro...,Disbursement timing for Title IV funds in subs...,single_hop_specifc_query_synthesizer
5,How do disbursement timing requirements differ...,[<1-hop>\n\nboth the credit or clock hours and...,In clock-hour or non-term credit-hour programs...,multi_hop_abstract_query_synthesizer
6,H0w doez the school determen if clinical work ...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Inclusion of clinical work in a standard term ...,multi_hop_abstract_query_synthesizer
7,How do clock-hour programs measure progress in...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Clock-hour programs measure progress in terms ...,multi_hop_abstract_query_synthesizer
8,How do Volume 8 and Volume 7 relate to the tim...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 8 discusses the inclusion of clinical w...,multi_hop_specific_query_synthesizer
9,Appendix A and B how relate to disbursement ti...,[<1-hop>\n\nboth the credit or clock hours and...,Appendix A explains how accelerated progressio...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)

### using default personas, refer to RAGAS docs for further info
### https://docs.ragas.io/en/latest/howtos/customizations/testgenerator/_persona_generator/#automatic-persona-generation
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10) 

In [20]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is a department in the context of academi...,"[Chapter 1 Academic Years, Academic Calendars,...",The provided context does not explicitly defin...,single_hop_specifc_query_synthesizer
1,Chapter 3 include clinical work?,[Inclusion of Clinical Work in a Standard Term...,Inclusion of Clinical Work in a Standard Term ...,single_hop_specifc_query_synthesizer
2,FWS is it like other programs with payment per...,[Non-Term Characteristics A program that measu...,The payment period is applicable to all Title ...,single_hop_specifc_query_synthesizer
3,How does the TEACH Grrant affect student disbu...,[both the credit or clock hours and the weeks ...,The TEACH Grant amount a student is eligible t...,single_hop_specifc_query_synthesizer
4,Wht criteria for nonstandard terms and non-ter...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The inclusion of clinical work in a standard t...,multi_hop_abstract_query_synthesizer
5,how timing and schedul constraints of clinical...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The context explains that clinical work includ...,multi_hop_abstract_query_synthesizer
6,how guidance and rules for disbursement and pa...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that disbursement timing ...,multi_hop_abstract_query_synthesizer
7,H0w do guidnace and priciples for detemining p...,[<1-hop>\n\nboth the credit or clock hours and...,The guidnace and priciples for detemining paym...,multi_hop_abstract_query_synthesizer
8,How do Appendix A and Appendix B collectively ...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix B provides detailed guidance on disbu...,multi_hop_specific_query_synthesizer
9,How do the definitions of academic years and i...,[<1-hop>\n\nboth the credit or clock hours and...,The definitions provided in Volume 2 and Volum...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context include:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (including student Federal PLUS Loans and parent Direct PLUS Loans on behalf of dependent students)  \n- Federal Stafford Loans (Subsidized and Unsubsidized, made under the FFEL Program before July 1, 2010)  \n- Federal SLS Loans  \n- Federal PLUS Loans (made under the FFEL Program before July 1, 2010)  \n- Direct Consolidation Loans  \n- Federal Consolidation Loans (made under the FFEL Program)\n\nAdditionally, Direct Subsidized Loans are available only to undergraduate students, while graduate or professional students are eligible only for Direct Unsubsidized Loans (and Direct PLUS Loans).'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

#### ✅ Answer:

- `qa_evaluator`: makes the LLM evaluate the response with the given query
- `labeled_helpfulness_evaluator`: labels the responses as helpful or not with the help of the LLM
- `empathy_evaluator`: checks if the responses have been written expressing empathy

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'respectful-selection-58' at:
https://smith.langchain.com/o/4a563880-75b7-483f-b9cd-cf740f81427b/datasets/69f2bc61-0902-4545-bfe7-bed32ec41dce/compare?selectedSessions=947f8046-1c6c-4546-b0a1-a7a59bf9268d




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 8 claculates clinical work in a standar...,I don't know.,None,Volume 8 provides guidance on including clinic...,0,0,0,0.901212,615c52d2-c300-401c-8fb4-33e60bed565a,66b60df4-ac69-421f-a59c-085a9eedbfcd
1,How does Volume 8 relate to the inclusion of c...,"Volume 8, specifically Chapter 3, provides add...",None,"Volume 8, Chapter 3, provides guidance on incl...",1,1,0,2.453351,d25cf64e-584f-4c80-9306-41476714ea84,a76992a1-7cb6-4539-9b37-fe46ad20378d
2,How do the definitions of academic years and i...,"Based on the provided context, the definitions...",None,The definitions provided in Volume 2 and Volum...,1,1,0,6.907918,52b1e740-078a-422e-a1c2-5626e742216c,4d9294e3-be9a-4f2a-b61d-ad7413ea8066
3,How do Appendix A and Appendix B collectively ...,I don't know.,None,Appendix B provides detailed guidance on disbu...,0,0,0,0.993591,68cd7b36-d231-4c47-8afb-1239246f4cc1,df52fc04-3745-4fbe-aa1c-1a0a0bdd6489
4,H0w do guidnace and priciples for detemining p...,"Based on the provided context, the guidance an...",None,The guidnace and priciples for detemining paym...,1,0,0,4.812429,3b99c15d-a035-481d-bf4e-76008df832ea,763066bc-a49e-4f26-90bc-f87cb8e4267c
5,how guidance and rules for disbursement and pa...,"Based on the provided context, guidance and ru...",None,The context explains that disbursement timing ...,1,1,0,14.408780,1096f576-c07a-40ed-8978-732c6c90b8cd,f2f994e3-3887-4922-80b1-f8e33ef6d3b0
6,how timing and schedul constraints of clinical...,I don't know.,None,The context explains that clinical work includ...,0,0,0,0.958463,c83ae08b-b73f-4a57-a15d-ade72fa2a21a,a1aa0f41-1d09-4c46-88f3-01501926545f
7,Wht criteria for nonstandard terms and non-ter...,Based on the provided context:\n\n**Criteria f...,None,The inclusion of clinical work in a standard t...,1,1,0,10.171585,032f2b22-81e9-427f-899e-51ce542ae720,4ef1e144-d85d-4dba-b903-47ed0832f238
8,How does the TEACH Grrant affect student disbu...,"Based on the provided context, the TEACH Grant...",None,The TEACH Grant amount a student is eligible t...,1,1,0,4.626318,afb50a8c-eef5-4534-b59d-06f85a2c659e,a2aa2118-8214-4d14-9bef-ff7c85e58106
9,FWS is it like other programs with payment per...,FWS (Federal Work-Study) is not like other Tit...,None,The payment period is applicable to all Title ...,1,1,0,1.714621,c192f46a-a2a0-445d-bbef-1a224cf455dc,a50643bb-57a3-4a76-987b-a4a1e771f48f


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### ✅ Answer:

The chunk size defines the number of characters to pick from the given corpus to form a chunk, there's pros and cons with large and smaller chunks. Especially when it comes to matching the cosine similarity between one or more chunks and the query or search tokens. Smaller number of chunks could mean less chances of semantic matching, larger chunks means more chances of semantic matching. As the performance of a system is quite dependent on the quality of the retrieval both in number and quality itself.

So in short we can say:
- _Small chunks_: Better precision, less noise, but may lose context and split related information
- _Large chunks_: More context and completeness, but more noise and harder to match specific queries
- _Sweet spot_: Balance between sufficient context and focused relevance

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### ✅ Answer:

The internal representation quality of an embeddings model has an impact on the quality of the response. Also depending on the closeness to the domain of a embeddings model it can perform better for one domain than another. Also different implementations of distance metric formula by different embeddings model can impact how it performs with any given corpus.

Some of the below ideas about embeddings should give us more insights on them:
- **Semantic understanding depth**: different models capture different levels of semantic nuance
- **Dimensionality effects**: higher dimensions can capture more information but may introduce noise
- **Training data domain**: models trained on domain-specific data perform better on similar content
- **Retrieval precision**: better embeddings = more accurate similarity matching = better retrieval

In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question! Based on the information provided in the context, there are several kinds of loans available to help cover college expenses:\n\n1. **Direct Subsidized Loans** – These loans are based on the student's financial need, and the government pays the interest while the student is in school.\n\n2. **Direct Unsubsidized Loans** – These are available to students regardless of financial need, and the student is responsible for interest payments during all periods.\n\n3. **Direct PLUS Loans** – These are loans that parents of dependent students can take out to cover the student's cost of attendance if they meet eligibility requirements. There is no fixed limit on these loans other than they cannot exceed the student's cost of attendance minus other financial aid received. Additionally, graduate or professional students can also apply for Direct PLUS Loans.\n\n4. **Additional Direct Unsubsidized Loans** – For independent students or dependent students whose parents are

Finally, we can evaluate the new chain on the same test set!

In [43]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'dear-order-55' at:
https://smith.langchain.com/o/4a563880-75b7-483f-b9cd-cf740f81427b/datasets/69f2bc61-0902-4545-bfe7-bed32ec41dce/compare?selectedSessions=621fe503-7fcd-40b8-83de-cfeee40cab58




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 8 claculates clinical work in a standar...,Thank you for your thoughtful question—it's gr...,None,Volume 8 provides guidance on including clinic...,1,1,1,5.301086,615c52d2-c300-401c-8fb4-33e60bed565a,d677b50f-4f1b-4278-a207-b5e1bfdc837d
1,How does Volume 8 relate to the inclusion of c...,Thank you for your thoughtful question. Based ...,None,"Volume 8, Chapter 3, provides guidance on incl...",1,0,1,6.543637,d25cf64e-584f-4c80-9306-41476714ea84,0412281c-d650-4f08-9cf2-aed54645c19e
2,How do the definitions of academic years and i...,Thank you for your thoughtful question. Based ...,None,The definitions provided in Volume 2 and Volum...,1,0,1,11.123948,52b1e740-078a-422e-a1c2-5626e742216c,8eeb1362-fd15-4472-87ff-03bdb707f7f7
3,How do Appendix A and Appendix B collectively ...,Thank you for your thoughtful question. It rea...,None,Appendix B provides detailed guidance on disbu...,0,0,1,3.653654,68cd7b36-d231-4c47-8afb-1239246f4cc1,ba4cb81d-b0f2-4584-b087-113efab23c24
4,H0w do guidnace and priciples for detemining p...,Thank you for your thoughtful question. Based ...,None,The guidnace and priciples for detemining paym...,1,1,1,4.009578,3b99c15d-a035-481d-bf4e-76008df832ea,dca01c95-65b3-4448-bbbc-3a8d1487cde3
5,how guidance and rules for disbursement and pa...,Thank you for your thoughtful question. Based ...,None,The context explains that disbursement timing ...,1,1,1,5.869165,1096f576-c07a-40ed-8978-732c6c90b8cd,71d1db9b-e2b7-4644-9cb2-15a72af5bf98
6,how timing and schedul constraints of clinical...,Thank you for your thoughtful question about h...,None,The context explains that clinical work includ...,1,0,1,5.132998,c83ae08b-b73f-4a57-a15d-ade72fa2a21a,35cb4d45-144b-41c6-bcc8-db0ac5533f7d
7,Wht criteria for nonstandard terms and non-ter...,Thank you for your thoughtful question. It sou...,None,The inclusion of clinical work in a standard t...,1,1,1,10.115492,032f2b22-81e9-427f-899e-51ce542ae720,428d8ce5-019a-4e58-888e-f72bd5fff793
8,How does the TEACH Grrant affect student disbu...,Thank you for your question. From the context ...,None,The TEACH Grant amount a student is eligible t...,1,1,1,3.432190,afb50a8c-eef5-4534-b59d-06f85a2c659e,105526c6-e1e8-406b-b676-ad2d83d8b3f6
9,FWS is it like other programs with payment per...,Thank you for your thoughtful question. Based ...,None,The payment period is applicable to all Title ...,1,1,1,4.144225,c192f46a-a2a0-445d-bbef-1a224cf455dc,dd061b3a-965b-42b3-a80f-6c6a66d782da


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

#### ✅ Answer:

![Basic versus Dopefied RAG comparison](Basic-versus-Dopefied-RAG.jpg)

The above screenshot shows the comparison between the Basic RAG system and the Dopefied RAG system. There's plus and minuses in the change between them:

- **Correctness** as compared to the Basic RAG **increased (0.20+ or >20%)** in the Dopefied RAG system 
- **Empathy** as compared to the Basic RAG **increased drastically (100%)** in the Dopefied RAG system, all responses had empathy in them
- **Helpfulness** as compared to the Basic RAG **decreased a bit (0.8x or ~8%)** in the Dopefied RAG system

![Side-by-side Basic versus Dopefied RAG comparison](./side-by-side-comparison.png)